In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE156309"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE156309"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE156309.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE156309.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE156309.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE156309_family.soft.gz', 'GSE156309_series_matrix.txt.gz']
Identified SOFT files: ['GSE156309_family.soft.gz']
Identified matrix files: ['GSE156309_series_matrix.txt.gz']



Background Information:
!Series_title	"Gene expression of 61 FFPE tissues of DLBCL patients at high-risk (aaIPI 2 or 3)"
!Series_summary	"Current staging classifications do not accurately predict the benefit of high-dose chemotherapy (HDC) with autologous stem-cell transplantation (ASCT) for patients with diffuse large B-cell lymphoma (DLBCL) at high risk (age-adjusted International Index [aaIPI] score 2 or 3), who have achieved first complete remission after R-CHOP (rituximab, cyclophosphamide, vincristine, doxorubicin, and prednisone) treatment. We aim to construct a genetic prognostic model for improving individualized risk stratification and response prediction for HDC/ASCT therapy. We identified differentially expressed mRNAs associated with relapse of DLBCL."
!Series_overall_design	"Affymetrix Human U133 Plus 2.0 microarrays (ThermoFisher Scientific, Waltham, MA, USA) identified differentially expressed mRNAs between 34 relapse and 27 relapse-free DLBCL patients."

Sample Charac

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from Affymetrix Human U133 Plus 2.0 microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: disease status is in row 3
trait_row = 3
# For age: age is in row 0
age_row = 0
# For gender: gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary (0 for relapse-free, 1 for relapse)"""
    if value is None:
        return None
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if "relapse-free" in value_str:
        return 0
    elif "relapse" in value_str:
        return 1
    return None

def convert_age(value):
    """Convert age value to continuous"""
    if value is None:
        return None
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    try:
        return float(value_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Since gender is not available, this function won't be used
    # but we define it as a placeholder
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data from matrix file properly
    matrix_file = os.path.join(in_cohort_dir, "GSE156309_series_matrix.txt.gz")
    
    # GEO series matrix files require special handling
    # First, read the file line by line to get the sample characteristics
    clinical_data = []
    with gzip.open(matrix_file, 'rt') as f:
        for line in f:
            if line.startswith('!Sample_characteristics_ch1'):
                clinical_data.append(line.strip().split('\t'))
            elif line.startswith('!Sample_geo_accession'):
                # This line contains the sample IDs
                sample_ids = line.strip().split('\t')[1:]
    
    # Convert to DataFrame
    clinical_df = pd.DataFrame(clinical_data)
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    clinical_preview = preview_df(selected_clinical)
    print("Clinical data preview:")
    print(clinical_preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{0: [nan, nan], 1: [0.0, nan], 2: [1.0, nan], 3: [0.0, nan], 4: [1.0, nan], 5: [1.0, nan], 6: [0.0, nan], 7: [1.0, nan], 8: [0.0, nan], 9: [1.0, nan], 10: [0.0, nan], 11: [0.0, nan], 12: [1.0, nan], 13: [1.0, nan], 14: [1.0, nan], 15: [1.0, nan], 16: [1.0, nan], 17: [1.0, nan], 18: [0.0, nan], 19: [1.0, nan], 20: [1.0, nan], 21: [1.0, nan], 22: [1.0, nan], 23: [1.0, nan], 24: [0.0, nan], 25: [0.0, nan], 26: [1.0, nan], 27: [1.0, nan], 28: [0.0, nan], 29: [0.0, nan], 30: [1.0, nan], 31: [1.0, nan], 32: [1.0, nan], 33: [0.0, nan], 34: [1.0, nan], 35: [0.0, nan], 36: [1.0, nan], 37: [1.0, nan], 38: [0.0, nan], 39: [1.0, nan], 40: [1.0, nan], 41: [1.0, nan], 42: [1.0, nan], 43: [0.0, nan], 44: [0.0, nan], 45: [0.0, nan], 46: [1.0, nan], 47: [0.0, nan], 48: [0.0, nan], 49: [1.0, nan], 50: [0.0, nan], 51: [1.0, nan], 52: [1.0, nan], 53: [0.0, nan], 54: [1.0, nan], 55: [0.0, nan], 56: [0.0, nan], 57: [0.0, nan], 58: [0.0, nan], 59: [1.0, nan], 60: [0.0, nan], 61: [0.0, 

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data shape: (54675, 61)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are in the format of Affymetrix probe IDs (like '1007_s_at'), not standard human gene symbols.
# They need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
           GSM4728797  GSM4728798  GSM4728799  GSM4728800  GSM4728801
ID                                                                   
1007_s_at   10.633903   10.853202    9.484531   10.025575   11.022525
1053_at      5.817172    5.000754    5.685826    4.819803    5.529107
117_at       9.109733    9.243769    5.650315    6.546760    9.065110
121_at      11.696432   12.256708   11.230113   10.986019   11.192433
1255_g_at    7.283554    6.124484    6.081273    8.422926    5.304300



Platform information:
!Series_title = Gene expression of 61 FFPE tissues of DLBCL patients at high-risk (aaIPI 2 or 3)
!Platform_title = [HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = June 03, 2009: annotation table updated with netaffx build 28
!Platform_description = June 06, 2012: annotation table updated with netaffx build 32
!Platform_description = June 23, 2016: annotation table updated with netaffx build 35
#Target Description =
#RefSeq Transcript ID = References to multiple sequences in RefSeq. The field contains the ID and Description for each entry, and there can be multiple entries per ProbeSet.
#Gene Ontology Biological Process = Gene Ontology Consortium Biological Process derived from LocusLink.  Each annotation consists of three parts


Gene annotation columns:
['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']

Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation store the probe identifiers and gene symbols
# From previous output, we can see:
# - 'ID' column contains probe identifiers that match gene_data index (e.g., '1007_s_at')
# - 'Gene Symbol' column contains the corresponding gene symbols (e.g., 'DDR1 /// MIR4640')

# 2. Get the gene mapping dataframe
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("Gene mapping preview:")
mapping_preview = preview_df(mapping_data)
print(mapping_preview)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

print("\nAfter mapping to gene symbols:")
print(f"Number of genes: {gene_data.shape[0]}")
print(f"Number of samples: {gene_data.shape[1]}")
print("First 10 gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



After mapping to gene symbols:
Number of genes: 21278
Number of samples: 61
First 10 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE156309.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (19845, 61)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE156309.csv
Loaded clinical data:
       1    2    3    4    5    6    7    8    9   10  ...   52   53   54  \
0                                                      ...                  
NaN  0.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  0.0  ...  1.0  0.0  1.0   
NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

      55   56   57   58   59   60   61  
0                                       
NaN  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 61 columns]
Number of common samples between clinical and genetic data: 0
Clinical data index: [nan, nan]
Gene data columns: ['GSM4728797', 'GSM4728798', 'GSM4728799', 'GSM4728800', 'GSM4728801', '...']
Extracted 61 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
            X-Linked_Lymphoproliferative_Syndrome
GSM4728797                          

Linked data shape after handling missing values: (61, 19846)
For the feature 'X-Linked_Lymphoproliferative_Syndrome', the least common label is '1' with 14 occurrences. This represents 22.95% of the dataset.
The distribution of the feature 'X-Linked_Lymphoproliferative_Syndrome' in this dataset is fine.

Is trait biased: False
A new JSON file was created at: ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json
Data quality check result: Usable


Linked data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE156309.csv
